In [1]:
s3_url = 's3a://dse-cohort5-group5/wildfire_capstone/integratedData.pca.parquet.gz'
pca_df = spark.read.parquet(s3_url)
pca_df.createOrReplaceTempView('pca')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1586972758394_0011,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
s3_url = 's3a://dse-cohort5-group5/wildfire_capstone/timeLaggedGridMetFeatures/partitioned/{agg}'
aggs = [
    'min',
    'max',
    'mean',
    'cumulative',
    'std',
]
dfs = []
table_names = []
base_df = spark.read.parquet('s3a://dse-cohort5-group5/wildfire_capstone/integratedData.renamed.parquet.gz')
table_names.append("fire_occurrences")
base_df.createOrReplaceTempView(table_names[0])
dfs.append(base_df)
for agg in aggs:
    print("Creating table:", agg)
    df = spark.read.parquet(s3_url.format(agg=agg))
    df.createOrReplaceTempView(agg)
    dfs.append(df)
    table_names.append(agg)

Creating table: min
Creating table: max
Creating table: mean
Creating table: cumulative
Creating table: std

In [3]:
base_df.printSchema()

root
 |-- date: long (nullable = true)
 |-- precipitation_amount_mm: double (nullable = true)
 |-- relative_humidity_%: double (nullable = true)
 |-- specific_humidity_kg/kg: double (nullable = true)
 |-- surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- wind_speed_m/s: double (nullable = true)
 |-- max_air_temperature_K: double (nullable = true)
 |-- min_air_temperature_K: double (nullable = true)
 |-- burning_index_g_Unitless: double (nullable = true)
 |-- dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- energy_release_component-g_Unitless: double (nullable = true)
 |-- potential_evapotranspiration_mm: double (nullable = true)
 |-- mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- fire_occurred: integer (nullable = true)
 |-- acres_burned: double (nullable = true)
 |-- fire_name: strin

In [4]:
base_df.show(1, vertical=True)

-RECORD 0---------------------------------------------------------------
 date                                             | 915148800000000000  
 precipitation_amount_mm                          | 0.0                 
 relative_humidity_%                              | 40.300000000000004  
 specific_humidity_kg/kg                          | 0.00589             
 surface_downwelling_shortwave_flux_in_air_W_m-2  | 138.0               
 wind_from_direction_Degrees_Clockwise_from_north | 123.0               
 wind_speed_m/s                                   | 1.6                 
 max_air_temperature_K                            | 293.1               
 min_air_temperature_K                            | 281.1               
 burning_index_g_Unitless                         | 24.0                
 dead_fuel_moisture_100hr_Percent                 | 16.0                
 dead_fuel_moisture_1000hr_Percent                | 15.5                
 energy_release_component-g_Unitless              |

In [5]:
dfs[1].printSchema()

root
 |-- 7d_min_precipitation_amount_mm: double (nullable = true)
 |-- 7d_min_relative_humidity_%: double (nullable = true)
 |-- 7d_min_specific_humidity_kg/kg: double (nullable = true)
 |-- 7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- 7d_min_wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- 7d_min_wind_speed_m/s: double (nullable = true)
 |-- 7d_min_max_air_temperature_K: double (nullable = true)
 |-- 7d_min_min_air_temperature_K: double (nullable = true)
 |-- 7d_min_burning_index_g_Unitless: double (nullable = true)
 |-- 7d_min_dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- 7d_min_dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- 7d_min_energy_release_component-g_Unitless: double (nullable = true)
 |-- 7d_min_potential_evapotranspiration_mm: double (nullable = true)
 |-- 7d_min_mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- 14d_min_precipitation_amount_mm: double (n

In [6]:
dfs[1].show(1, vertical=True)

-RECORD 0-------------------------------------------------------------------------
 7d_min_precipitation_amount_mm                            | 0.0                  
 7d_min_relative_humidity_%                                | 28.0                 
 7d_min_specific_humidity_kg/kg                            | 0.00583              
 7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2    | 116.30000000000001   
 7d_min_wind_from_direction_Degrees_Clockwise_from_north   | 112.0                
 7d_min_wind_speed_m/s                                     | 0.6000000000000001   
 7d_min_max_air_temperature_K                              | 291.2                
 7d_min_min_air_temperature_K                              | 281.4                
 7d_min_burning_index_g_Unitless                           | 21.0                 
 7d_min_dead_fuel_moisture_100hr_Percent                   | 14.700000000000001   
 7d_min_dead_fuel_moisture_1000hr_Percent                  | 13.9                 
 7d_

In [7]:
pca_df.printSchema()

root
 |-- date: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pcaFeatures: vector (nullable = true)

In [8]:
pca_df.show(1, vertical=True)

-RECORD 0---------------------------
 date        | 915235200000000000   
 latitude    | 32.608333333333334   
 longitude   | -116.93333330000002  
 pcaFeatures | [-18.953460978093... 
only showing top 1 row

In [9]:
original_cols = [table_names[0] + ".`{}`".format(c) for c in base_df.schema.names[1:-6]]
for i, df in enumerate(dfs[1:], 1):
    original_cols += [table_names[i] + '.`{}`'.format(c) for c in df.schema.names[:-4]]
original_cols

['fire_occurrences.`precipitation_amount_mm`', 'fire_occurrences.`relative_humidity_%`', 'fire_occurrences.`specific_humidity_kg/kg`', 'fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`', 'fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`', 'fire_occurrences.`wind_speed_m/s`', 'fire_occurrences.`max_air_temperature_K`', 'fire_occurrences.`min_air_temperature_K`', 'fire_occurrences.`burning_index_g_Unitless`', 'fire_occurrences.`dead_fuel_moisture_100hr_Percent`', 'fire_occurrences.`dead_fuel_moisture_1000hr_Percent`', 'fire_occurrences.`energy_release_component-g_Unitless`', 'fire_occurrences.`potential_evapotranspiration_mm`', 'fire_occurrences.`mean_vapor_pressure_deficit_kPa`', 'min.`7d_min_precipitation_amount_mm`', 'min.`7d_min_relative_humidity_%`', 'min.`7d_min_specific_humidity_kg/kg`', 'min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`7d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`7d_min_wind_speed_m/s`',

In [10]:
select_template = """
SELECT fire_occurrences.date, fire_occurrences.latitude, fire_occurrences.longitude, {pca_cols}
FROM fire_occurrences, {tables} WHERE """
where_template = """
to_unix_timestamp({table}.date) = cast(fire_occurrences.date/1e9 as long)
AND {table}.latitude = fire_occurrences.latitude
AND {table}.longitude = fire_occurrences.longitude"""

query = select_template.format(tables=", ".join(aggs),
                               pca_cols=", ".join(original_cols)) \
        + " AND ".join(where_template.format(table=agg) for agg in aggs)
query

'\nSELECT fire_occurrences.date, fire_occurrences.latitude, fire_occurrences.longitude, fire_occurrences.`precipitation_amount_mm`, fire_occurrences.`relative_humidity_%`, fire_occurrences.`specific_humidity_kg/kg`, fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`, fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`, fire_occurrences.`wind_speed_m/s`, fire_occurrences.`max_air_temperature_K`, fire_occurrences.`min_air_temperature_K`, fire_occurrences.`burning_index_g_Unitless`, fire_occurrences.`dead_fuel_moisture_100hr_Percent`, fire_occurrences.`dead_fuel_moisture_1000hr_Percent`, fire_occurrences.`energy_release_component-g_Unitless`, fire_occurrences.`potential_evapotranspiration_mm`, fire_occurrences.`mean_vapor_pressure_deficit_kPa`, min.`7d_min_precipitation_amount_mm`, min.`7d_min_relative_humidity_%`, min.`7d_min_specific_humidity_kg/kg`, min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`, min.`7d_min_wind_from_direction_Degrees_Cloc

In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[c.split('.')[1].replace('`', '') for c in original_cols], outputCol="features")
joined = spark.sql(query)
joined = assembler.setHandleInvalid("skip").transform(joined)
joined.createOrReplaceTempView("joined")
joined.cache()

DataFrame[date: bigint, latitude: double, longitude: double, precipitation_amount_mm: double, relative_humidity_%: double, specific_humidity_kg/kg: double, surface_downwelling_shortwave_flux_in_air_W_m-2: double, wind_from_direction_Degrees_Clockwise_from_north: double, wind_speed_m/s: double, max_air_temperature_K: double, min_air_temperature_K: double, burning_index_g_Unitless: double, dead_fuel_moisture_100hr_Percent: double, dead_fuel_moisture_1000hr_Percent: double, energy_release_component-g_Unitless: double, potential_evapotranspiration_mm: double, mean_vapor_pressure_deficit_kPa: double, 7d_min_precipitation_amount_mm: double, 7d_min_relative_humidity_%: double, 7d_min_specific_humidity_kg/kg: double, 7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2: double, 7d_min_wind_from_direction_Degrees_Clockwise_from_north: double, 7d_min_wind_speed_m/s: double, 7d_min_max_air_temperature_K: double, 7d_min_min_air_temperature_K: double, 7d_min_burning_index_g_Unitless: double, 7d_m

In [12]:
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

concat = F.udf(lambda v1, v2: Vectors.dense(list(v1) + list(v2)[:20]), VectorUDT())

concated = spark.sql("""
SELECT joined.features, pca.pcaFeatures
FROM joined, pca
WHERE joined.latitude = pca.latitude AND joined.longitude = pca.longitude AND joined.date = pca.date""")\
    .select(concat(joined.features, pca_df.pcaFeatures).alias('merged'))
concated.cache()

DataFrame[merged: vector]

In [13]:
concated.printSchema()

root
 |-- merged: vector (nullable = true)

In [14]:
concated.show(1, vertical=True)

-RECORD 0----------------------
 merged | [1.20000000000000... 
only showing top 1 row

In [15]:
row = concated.rdd.take(1)[0][0]
print(row.shape)

(594,)

In [16]:
from pyspark.ml.stat import Correlation

r1 = Correlation.corr(concated, "merged")
r1.cache()

DataFrame[pearson(merged): matrix]

In [17]:
r1.printSchema()

root
 |-- pearson(merged): matrix (nullable = false)

In [18]:
r1.collect()[0][0]

DenseMatrix(594, 594, [1.0, 0.2699, 0.006, -0.2784, -0.0431, 0.3169, -0.253, -0.1351, ..., 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0], False)

In [19]:
corr = r1.collect()[0][0].toArray()

In [29]:
import numpy as np
low = 594 - 20
pca_idxs = list(range(low, 594))
top5_correlates = {}
for pca_idx in pca_idxs:
    top5 = np.argsort(np.abs(np.nan_to_num(corr[pca_idx])[:low]))[-1:-6:-1]
    print(corr[pca_idx][top5])
    top5_correlates[pca_idx] = top5
top5_correlates

[ 0.94104098  0.94065001  0.939614    0.93916116 -0.93712351]
[0.91163888 0.90975966 0.90342582 0.90039475 0.89930836]
[-0.75534748  0.74715444  0.74190775  0.71848208  0.71464701]
[-0.59157768 -0.58739117 -0.57559074 -0.57307921 -0.54366682]
[0.63370628 0.61030346 0.5927075  0.58959805 0.5226046 ]
[0.55248368 0.54932503 0.5413866  0.53448398 0.52604457]
[-0.58402447 -0.50604175 -0.43501007 -0.42683517 -0.42084658]
[-0.47036274 -0.45238763 -0.45228269  0.42701988 -0.40088067]
[ 0.52281856  0.50112902  0.47363703  0.44918608 -0.41419608]
[ 0.54297905  0.49504995 -0.46322699  0.4167323  -0.41602895]
[-0.34048446 -0.33441578 -0.32665587  0.30263283 -0.29476438]
[0.3384111  0.33838254 0.33224193 0.33150833 0.28378455]
[ 0.40007302  0.33689424 -0.29952909  0.27668889 -0.27199255]
[ 0.40095248  0.35891341 -0.32736765  0.30276121 -0.2990228 ]
[-0.29818391  0.28421662  0.27861194 -0.27294288  0.27041054]
[ 0.42615037 -0.30732571 -0.29358446  0.29282406 -0.29268097]
[ 0.25956746  0.24109349 -0.

In [30]:
col_names = ['fire_occurrences.`precipitation_amount_mm`', 'fire_occurrences.`relative_humidity_%`', 'fire_occurrences.`specific_humidity_kg/kg`', 'fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`', 'fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`', 'fire_occurrences.`wind_speed_m/s`', 'fire_occurrences.`max_air_temperature_K`', 'fire_occurrences.`min_air_temperature_K`', 'fire_occurrences.`burning_index_g_Unitless`', 'fire_occurrences.`dead_fuel_moisture_100hr_Percent`', 'fire_occurrences.`dead_fuel_moisture_1000hr_Percent`', 'fire_occurrences.`energy_release_component-g_Unitless`', 'fire_occurrences.`potential_evapotranspiration_mm`', 'fire_occurrences.`mean_vapor_pressure_deficit_kPa`', 'min.`7d_min_precipitation_amount_mm`', 'min.`7d_min_relative_humidity_%`', 'min.`7d_min_specific_humidity_kg/kg`', 'min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`7d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`7d_min_wind_speed_m/s`', 'min.`7d_min_max_air_temperature_K`', 'min.`7d_min_min_air_temperature_K`', 'min.`7d_min_burning_index_g_Unitless`', 'min.`7d_min_dead_fuel_moisture_100hr_Percent`', 'min.`7d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`7d_min_energy_release_component-g_Unitless`', 'min.`7d_min_potential_evapotranspiration_mm`', 'min.`7d_min_mean_vapor_pressure_deficit_kPa`', 'min.`14d_min_precipitation_amount_mm`', 'min.`14d_min_relative_humidity_%`', 'min.`14d_min_specific_humidity_kg/kg`', 'min.`14d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`14d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`14d_min_wind_speed_m/s`', 'min.`14d_min_max_air_temperature_K`', 'min.`14d_min_min_air_temperature_K`', 'min.`14d_min_burning_index_g_Unitless`', 'min.`14d_min_dead_fuel_moisture_100hr_Percent`', 'min.`14d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`14d_min_energy_release_component-g_Unitless`', 'min.`14d_min_potential_evapotranspiration_mm`', 'min.`14d_min_mean_vapor_pressure_deficit_kPa`', 'min.`21d_min_precipitation_amount_mm`', 'min.`21d_min_relative_humidity_%`', 'min.`21d_min_specific_humidity_kg/kg`', 'min.`21d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`21d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`21d_min_wind_speed_m/s`', 'min.`21d_min_max_air_temperature_K`', 'min.`21d_min_min_air_temperature_K`', 'min.`21d_min_burning_index_g_Unitless`', 'min.`21d_min_dead_fuel_moisture_100hr_Percent`', 'min.`21d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`21d_min_energy_release_component-g_Unitless`', 'min.`21d_min_potential_evapotranspiration_mm`', 'min.`21d_min_mean_vapor_pressure_deficit_kPa`', 'min.`30d_min_precipitation_amount_mm`', 'min.`30d_min_relative_humidity_%`', 'min.`30d_min_specific_humidity_kg/kg`', 'min.`30d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`30d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`30d_min_wind_speed_m/s`', 'min.`30d_min_max_air_temperature_K`', 'min.`30d_min_min_air_temperature_K`', 'min.`30d_min_burning_index_g_Unitless`', 'min.`30d_min_dead_fuel_moisture_100hr_Percent`', 'min.`30d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`30d_min_energy_release_component-g_Unitless`', 'min.`30d_min_potential_evapotranspiration_mm`', 'min.`30d_min_mean_vapor_pressure_deficit_kPa`', 'min.`60d_min_precipitation_amount_mm`', 'min.`60d_min_relative_humidity_%`', 'min.`60d_min_specific_humidity_kg/kg`', 'min.`60d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`60d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`60d_min_wind_speed_m/s`', 'min.`60d_min_max_air_temperature_K`', 'min.`60d_min_min_air_temperature_K`', 'min.`60d_min_burning_index_g_Unitless`', 'min.`60d_min_dead_fuel_moisture_100hr_Percent`', 'min.`60d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`60d_min_energy_release_component-g_Unitless`', 'min.`60d_min_potential_evapotranspiration_mm`', 'min.`60d_min_mean_vapor_pressure_deficit_kPa`', 'min.`90d_min_precipitation_amount_mm`', 'min.`90d_min_relative_humidity_%`', 'min.`90d_min_specific_humidity_kg/kg`', 'min.`90d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`90d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`90d_min_wind_speed_m/s`', 'min.`90d_min_max_air_temperature_K`', 'min.`90d_min_min_air_temperature_K`', 'min.`90d_min_burning_index_g_Unitless`', 'min.`90d_min_dead_fuel_moisture_100hr_Percent`', 'min.`90d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`90d_min_energy_release_component-g_Unitless`', 'min.`90d_min_potential_evapotranspiration_mm`', 'min.`90d_min_mean_vapor_pressure_deficit_kPa`', 'min.`180d_min_precipitation_amount_mm`', 'min.`180d_min_relative_humidity_%`', 'min.`180d_min_specific_humidity_kg/kg`', 'min.`180d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`180d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`180d_min_wind_speed_m/s`', 'min.`180d_min_max_air_temperature_K`', 'min.`180d_min_min_air_temperature_K`', 'min.`180d_min_burning_index_g_Unitless`', 'min.`180d_min_dead_fuel_moisture_100hr_Percent`', 'min.`180d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`180d_min_energy_release_component-g_Unitless`', 'min.`180d_min_potential_evapotranspiration_mm`', 'min.`180d_min_mean_vapor_pressure_deficit_kPa`', 'min.`365d_min_precipitation_amount_mm`', 'min.`365d_min_relative_humidity_%`', 'min.`365d_min_specific_humidity_kg/kg`', 'min.`365d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`365d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`365d_min_wind_speed_m/s`', 'min.`365d_min_max_air_temperature_K`', 'min.`365d_min_min_air_temperature_K`', 'min.`365d_min_burning_index_g_Unitless`', 'min.`365d_min_dead_fuel_moisture_100hr_Percent`', 'min.`365d_min_dead_fuel_moisture_1000hr_Percent`', 'min.`365d_min_energy_release_component-g_Unitless`', 'min.`365d_min_potential_evapotranspiration_mm`', 'min.`365d_min_mean_vapor_pressure_deficit_kPa`', 'max.`7d_max_precipitation_amount_mm`', 'max.`7d_max_relative_humidity_%`', 'max.`7d_max_specific_humidity_kg/kg`', 'max.`7d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`7d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`7d_max_wind_speed_m/s`', 'max.`7d_max_max_air_temperature_K`', 'max.`7d_max_min_air_temperature_K`', 'max.`7d_max_burning_index_g_Unitless`', 'max.`7d_max_dead_fuel_moisture_100hr_Percent`', 'max.`7d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`7d_max_energy_release_component-g_Unitless`', 'max.`7d_max_potential_evapotranspiration_mm`', 'max.`7d_max_mean_vapor_pressure_deficit_kPa`', 'max.`14d_max_precipitation_amount_mm`', 'max.`14d_max_relative_humidity_%`', 'max.`14d_max_specific_humidity_kg/kg`', 'max.`14d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`14d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`14d_max_wind_speed_m/s`', 'max.`14d_max_max_air_temperature_K`', 'max.`14d_max_min_air_temperature_K`', 'max.`14d_max_burning_index_g_Unitless`', 'max.`14d_max_dead_fuel_moisture_100hr_Percent`', 'max.`14d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`14d_max_energy_release_component-g_Unitless`', 'max.`14d_max_potential_evapotranspiration_mm`', 'max.`14d_max_mean_vapor_pressure_deficit_kPa`', 'max.`21d_max_precipitation_amount_mm`', 'max.`21d_max_relative_humidity_%`', 'max.`21d_max_specific_humidity_kg/kg`', 'max.`21d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`21d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`21d_max_wind_speed_m/s`', 'max.`21d_max_max_air_temperature_K`', 'max.`21d_max_min_air_temperature_K`', 'max.`21d_max_burning_index_g_Unitless`', 'max.`21d_max_dead_fuel_moisture_100hr_Percent`', 'max.`21d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`21d_max_energy_release_component-g_Unitless`', 'max.`21d_max_potential_evapotranspiration_mm`', 'max.`21d_max_mean_vapor_pressure_deficit_kPa`', 'max.`30d_max_precipitation_amount_mm`', 'max.`30d_max_relative_humidity_%`', 'max.`30d_max_specific_humidity_kg/kg`', 'max.`30d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`30d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`30d_max_wind_speed_m/s`', 'max.`30d_max_max_air_temperature_K`', 'max.`30d_max_min_air_temperature_K`', 'max.`30d_max_burning_index_g_Unitless`', 'max.`30d_max_dead_fuel_moisture_100hr_Percent`', 'max.`30d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`30d_max_energy_release_component-g_Unitless`', 'max.`30d_max_potential_evapotranspiration_mm`', 'max.`30d_max_mean_vapor_pressure_deficit_kPa`', 'max.`60d_max_precipitation_amount_mm`', 'max.`60d_max_relative_humidity_%`', 'max.`60d_max_specific_humidity_kg/kg`', 'max.`60d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`60d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`60d_max_wind_speed_m/s`', 'max.`60d_max_max_air_temperature_K`', 'max.`60d_max_min_air_temperature_K`', 'max.`60d_max_burning_index_g_Unitless`', 'max.`60d_max_dead_fuel_moisture_100hr_Percent`', 'max.`60d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`60d_max_energy_release_component-g_Unitless`', 'max.`60d_max_potential_evapotranspiration_mm`', 'max.`60d_max_mean_vapor_pressure_deficit_kPa`', 'max.`90d_max_precipitation_amount_mm`', 'max.`90d_max_relative_humidity_%`', 'max.`90d_max_specific_humidity_kg/kg`', 'max.`90d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`90d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`90d_max_wind_speed_m/s`', 'max.`90d_max_max_air_temperature_K`', 'max.`90d_max_min_air_temperature_K`', 'max.`90d_max_burning_index_g_Unitless`', 'max.`90d_max_dead_fuel_moisture_100hr_Percent`', 'max.`90d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`90d_max_energy_release_component-g_Unitless`', 'max.`90d_max_potential_evapotranspiration_mm`', 'max.`90d_max_mean_vapor_pressure_deficit_kPa`', 'max.`180d_max_precipitation_amount_mm`', 'max.`180d_max_relative_humidity_%`', 'max.`180d_max_specific_humidity_kg/kg`', 'max.`180d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`180d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`180d_max_wind_speed_m/s`', 'max.`180d_max_max_air_temperature_K`', 'max.`180d_max_min_air_temperature_K`', 'max.`180d_max_burning_index_g_Unitless`', 'max.`180d_max_dead_fuel_moisture_100hr_Percent`', 'max.`180d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`180d_max_energy_release_component-g_Unitless`', 'max.`180d_max_potential_evapotranspiration_mm`', 'max.`180d_max_mean_vapor_pressure_deficit_kPa`', 'max.`365d_max_precipitation_amount_mm`', 'max.`365d_max_relative_humidity_%`', 'max.`365d_max_specific_humidity_kg/kg`', 'max.`365d_max_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'max.`365d_max_wind_from_direction_Degrees_Clockwise_from_north`', 'max.`365d_max_wind_speed_m/s`', 'max.`365d_max_max_air_temperature_K`', 'max.`365d_max_min_air_temperature_K`', 'max.`365d_max_burning_index_g_Unitless`', 'max.`365d_max_dead_fuel_moisture_100hr_Percent`', 'max.`365d_max_dead_fuel_moisture_1000hr_Percent`', 'max.`365d_max_energy_release_component-g_Unitless`', 'max.`365d_max_potential_evapotranspiration_mm`', 'max.`365d_max_mean_vapor_pressure_deficit_kPa`', 'mean.`7d_mean_precipitation_amount_mm`', 'mean.`7d_mean_relative_humidity_%`', 'mean.`7d_mean_specific_humidity_kg/kg`', 'mean.`7d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`7d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`7d_mean_wind_speed_m/s`', 'mean.`7d_mean_max_air_temperature_K`', 'mean.`7d_mean_min_air_temperature_K`', 'mean.`7d_mean_burning_index_g_Unitless`', 'mean.`7d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`7d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`7d_mean_energy_release_component-g_Unitless`', 'mean.`7d_mean_potential_evapotranspiration_mm`', 'mean.`7d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`14d_mean_precipitation_amount_mm`', 'mean.`14d_mean_relative_humidity_%`', 'mean.`14d_mean_specific_humidity_kg/kg`', 'mean.`14d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`14d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`14d_mean_wind_speed_m/s`', 'mean.`14d_mean_max_air_temperature_K`', 'mean.`14d_mean_min_air_temperature_K`', 'mean.`14d_mean_burning_index_g_Unitless`', 'mean.`14d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`14d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`14d_mean_energy_release_component-g_Unitless`', 'mean.`14d_mean_potential_evapotranspiration_mm`', 'mean.`14d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`21d_mean_precipitation_amount_mm`', 'mean.`21d_mean_relative_humidity_%`', 'mean.`21d_mean_specific_humidity_kg/kg`', 'mean.`21d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`21d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`21d_mean_wind_speed_m/s`', 'mean.`21d_mean_max_air_temperature_K`', 'mean.`21d_mean_min_air_temperature_K`', 'mean.`21d_mean_burning_index_g_Unitless`', 'mean.`21d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`21d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`21d_mean_energy_release_component-g_Unitless`', 'mean.`21d_mean_potential_evapotranspiration_mm`', 'mean.`21d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`30d_mean_precipitation_amount_mm`', 'mean.`30d_mean_relative_humidity_%`', 'mean.`30d_mean_specific_humidity_kg/kg`', 'mean.`30d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`30d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`30d_mean_wind_speed_m/s`', 'mean.`30d_mean_max_air_temperature_K`', 'mean.`30d_mean_min_air_temperature_K`', 'mean.`30d_mean_burning_index_g_Unitless`', 'mean.`30d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`30d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`30d_mean_energy_release_component-g_Unitless`', 'mean.`30d_mean_potential_evapotranspiration_mm`', 'mean.`30d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`60d_mean_precipitation_amount_mm`', 'mean.`60d_mean_relative_humidity_%`', 'mean.`60d_mean_specific_humidity_kg/kg`', 'mean.`60d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`60d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`60d_mean_wind_speed_m/s`', 'mean.`60d_mean_max_air_temperature_K`', 'mean.`60d_mean_min_air_temperature_K`', 'mean.`60d_mean_burning_index_g_Unitless`', 'mean.`60d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`60d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`60d_mean_energy_release_component-g_Unitless`', 'mean.`60d_mean_potential_evapotranspiration_mm`', 'mean.`60d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`90d_mean_precipitation_amount_mm`', 'mean.`90d_mean_relative_humidity_%`', 'mean.`90d_mean_specific_humidity_kg/kg`', 'mean.`90d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`90d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`90d_mean_wind_speed_m/s`', 'mean.`90d_mean_max_air_temperature_K`', 'mean.`90d_mean_min_air_temperature_K`', 'mean.`90d_mean_burning_index_g_Unitless`', 'mean.`90d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`90d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`90d_mean_energy_release_component-g_Unitless`', 'mean.`90d_mean_potential_evapotranspiration_mm`', 'mean.`90d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`180d_mean_precipitation_amount_mm`', 'mean.`180d_mean_relative_humidity_%`', 'mean.`180d_mean_specific_humidity_kg/kg`', 'mean.`180d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`180d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`180d_mean_wind_speed_m/s`', 'mean.`180d_mean_max_air_temperature_K`', 'mean.`180d_mean_min_air_temperature_K`', 'mean.`180d_mean_burning_index_g_Unitless`', 'mean.`180d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`180d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`180d_mean_energy_release_component-g_Unitless`', 'mean.`180d_mean_potential_evapotranspiration_mm`', 'mean.`180d_mean_mean_vapor_pressure_deficit_kPa`', 'mean.`365d_mean_precipitation_amount_mm`', 'mean.`365d_mean_relative_humidity_%`', 'mean.`365d_mean_specific_humidity_kg/kg`', 'mean.`365d_mean_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'mean.`365d_mean_wind_from_direction_Degrees_Clockwise_from_north`', 'mean.`365d_mean_wind_speed_m/s`', 'mean.`365d_mean_max_air_temperature_K`', 'mean.`365d_mean_min_air_temperature_K`', 'mean.`365d_mean_burning_index_g_Unitless`', 'mean.`365d_mean_dead_fuel_moisture_100hr_Percent`', 'mean.`365d_mean_dead_fuel_moisture_1000hr_Percent`', 'mean.`365d_mean_energy_release_component-g_Unitless`', 'mean.`365d_mean_potential_evapotranspiration_mm`', 'mean.`365d_mean_mean_vapor_pressure_deficit_kPa`', 'cumulative.`7d_cumulative_precipitation_amount_mm`', 'cumulative.`7d_cumulative_relative_humidity_%`', 'cumulative.`7d_cumulative_specific_humidity_kg/kg`', 'cumulative.`7d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`7d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`7d_cumulative_wind_speed_m/s`', 'cumulative.`7d_cumulative_max_air_temperature_K`', 'cumulative.`7d_cumulative_min_air_temperature_K`', 'cumulative.`7d_cumulative_burning_index_g_Unitless`', 'cumulative.`7d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`7d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`7d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`7d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`7d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`14d_cumulative_precipitation_amount_mm`', 'cumulative.`14d_cumulative_relative_humidity_%`', 'cumulative.`14d_cumulative_specific_humidity_kg/kg`', 'cumulative.`14d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`14d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`14d_cumulative_wind_speed_m/s`', 'cumulative.`14d_cumulative_max_air_temperature_K`', 'cumulative.`14d_cumulative_min_air_temperature_K`', 'cumulative.`14d_cumulative_burning_index_g_Unitless`', 'cumulative.`14d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`14d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`14d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`14d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`14d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`21d_cumulative_precipitation_amount_mm`', 'cumulative.`21d_cumulative_relative_humidity_%`', 'cumulative.`21d_cumulative_specific_humidity_kg/kg`', 'cumulative.`21d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`21d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`21d_cumulative_wind_speed_m/s`', 'cumulative.`21d_cumulative_max_air_temperature_K`', 'cumulative.`21d_cumulative_min_air_temperature_K`', 'cumulative.`21d_cumulative_burning_index_g_Unitless`', 'cumulative.`21d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`21d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`21d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`21d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`21d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`30d_cumulative_precipitation_amount_mm`', 'cumulative.`30d_cumulative_relative_humidity_%`', 'cumulative.`30d_cumulative_specific_humidity_kg/kg`', 'cumulative.`30d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`30d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`30d_cumulative_wind_speed_m/s`', 'cumulative.`30d_cumulative_max_air_temperature_K`', 'cumulative.`30d_cumulative_min_air_temperature_K`', 'cumulative.`30d_cumulative_burning_index_g_Unitless`', 'cumulative.`30d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`30d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`30d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`30d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`30d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`60d_cumulative_precipitation_amount_mm`', 'cumulative.`60d_cumulative_relative_humidity_%`', 'cumulative.`60d_cumulative_specific_humidity_kg/kg`', 'cumulative.`60d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`60d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`60d_cumulative_wind_speed_m/s`', 'cumulative.`60d_cumulative_max_air_temperature_K`', 'cumulative.`60d_cumulative_min_air_temperature_K`', 'cumulative.`60d_cumulative_burning_index_g_Unitless`', 'cumulative.`60d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`60d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`60d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`60d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`60d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`90d_cumulative_precipitation_amount_mm`', 'cumulative.`90d_cumulative_relative_humidity_%`', 'cumulative.`90d_cumulative_specific_humidity_kg/kg`', 'cumulative.`90d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`90d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`90d_cumulative_wind_speed_m/s`', 'cumulative.`90d_cumulative_max_air_temperature_K`', 'cumulative.`90d_cumulative_min_air_temperature_K`', 'cumulative.`90d_cumulative_burning_index_g_Unitless`', 'cumulative.`90d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`90d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`90d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`90d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`90d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`180d_cumulative_precipitation_amount_mm`', 'cumulative.`180d_cumulative_relative_humidity_%`', 'cumulative.`180d_cumulative_specific_humidity_kg/kg`', 'cumulative.`180d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`180d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`180d_cumulative_wind_speed_m/s`', 'cumulative.`180d_cumulative_max_air_temperature_K`', 'cumulative.`180d_cumulative_min_air_temperature_K`', 'cumulative.`180d_cumulative_burning_index_g_Unitless`', 'cumulative.`180d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`180d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`180d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`180d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`180d_cumulative_mean_vapor_pressure_deficit_kPa`', 'cumulative.`365d_cumulative_precipitation_amount_mm`', 'cumulative.`365d_cumulative_relative_humidity_%`', 'cumulative.`365d_cumulative_specific_humidity_kg/kg`', 'cumulative.`365d_cumulative_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'cumulative.`365d_cumulative_wind_from_direction_Degrees_Clockwise_from_north`', 'cumulative.`365d_cumulative_wind_speed_m/s`', 'cumulative.`365d_cumulative_max_air_temperature_K`', 'cumulative.`365d_cumulative_min_air_temperature_K`', 'cumulative.`365d_cumulative_burning_index_g_Unitless`', 'cumulative.`365d_cumulative_dead_fuel_moisture_100hr_Percent`', 'cumulative.`365d_cumulative_dead_fuel_moisture_1000hr_Percent`', 'cumulative.`365d_cumulative_energy_release_component-g_Unitless`', 'cumulative.`365d_cumulative_potential_evapotranspiration_mm`', 'cumulative.`365d_cumulative_mean_vapor_pressure_deficit_kPa`', 'std.`7d_std_precipitation_amount_mm`', 'std.`7d_std_relative_humidity_%`', 'std.`7d_std_specific_humidity_kg/kg`', 'std.`7d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`7d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`7d_std_wind_speed_m/s`', 'std.`7d_std_max_air_temperature_K`', 'std.`7d_std_min_air_temperature_K`', 'std.`7d_std_burning_index_g_Unitless`', 'std.`7d_std_dead_fuel_moisture_100hr_Percent`', 'std.`7d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`7d_std_energy_release_component-g_Unitless`', 'std.`7d_std_potential_evapotranspiration_mm`', 'std.`7d_std_mean_vapor_pressure_deficit_kPa`', 'std.`14d_std_precipitation_amount_mm`', 'std.`14d_std_relative_humidity_%`', 'std.`14d_std_specific_humidity_kg/kg`', 'std.`14d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`14d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`14d_std_wind_speed_m/s`', 'std.`14d_std_max_air_temperature_K`', 'std.`14d_std_min_air_temperature_K`', 'std.`14d_std_burning_index_g_Unitless`', 'std.`14d_std_dead_fuel_moisture_100hr_Percent`', 'std.`14d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`14d_std_energy_release_component-g_Unitless`', 'std.`14d_std_potential_evapotranspiration_mm`', 'std.`14d_std_mean_vapor_pressure_deficit_kPa`', 'std.`21d_std_precipitation_amount_mm`', 'std.`21d_std_relative_humidity_%`', 'std.`21d_std_specific_humidity_kg/kg`', 'std.`21d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`21d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`21d_std_wind_speed_m/s`', 'std.`21d_std_max_air_temperature_K`', 'std.`21d_std_min_air_temperature_K`', 'std.`21d_std_burning_index_g_Unitless`', 'std.`21d_std_dead_fuel_moisture_100hr_Percent`', 'std.`21d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`21d_std_energy_release_component-g_Unitless`', 'std.`21d_std_potential_evapotranspiration_mm`', 'std.`21d_std_mean_vapor_pressure_deficit_kPa`', 'std.`30d_std_precipitation_amount_mm`', 'std.`30d_std_relative_humidity_%`', 'std.`30d_std_specific_humidity_kg/kg`', 'std.`30d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`30d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`30d_std_wind_speed_m/s`', 'std.`30d_std_max_air_temperature_K`', 'std.`30d_std_min_air_temperature_K`', 'std.`30d_std_burning_index_g_Unitless`', 'std.`30d_std_dead_fuel_moisture_100hr_Percent`', 'std.`30d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`30d_std_energy_release_component-g_Unitless`', 'std.`30d_std_potential_evapotranspiration_mm`', 'std.`30d_std_mean_vapor_pressure_deficit_kPa`', 'std.`60d_std_precipitation_amount_mm`', 'std.`60d_std_relative_humidity_%`', 'std.`60d_std_specific_humidity_kg/kg`', 'std.`60d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`60d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`60d_std_wind_speed_m/s`', 'std.`60d_std_max_air_temperature_K`', 'std.`60d_std_min_air_temperature_K`', 'std.`60d_std_burning_index_g_Unitless`', 'std.`60d_std_dead_fuel_moisture_100hr_Percent`', 'std.`60d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`60d_std_energy_release_component-g_Unitless`', 'std.`60d_std_potential_evapotranspiration_mm`', 'std.`60d_std_mean_vapor_pressure_deficit_kPa`', 'std.`90d_std_precipitation_amount_mm`', 'std.`90d_std_relative_humidity_%`', 'std.`90d_std_specific_humidity_kg/kg`', 'std.`90d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`90d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`90d_std_wind_speed_m/s`', 'std.`90d_std_max_air_temperature_K`', 'std.`90d_std_min_air_temperature_K`', 'std.`90d_std_burning_index_g_Unitless`', 'std.`90d_std_dead_fuel_moisture_100hr_Percent`', 'std.`90d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`90d_std_energy_release_component-g_Unitless`', 'std.`90d_std_potential_evapotranspiration_mm`', 'std.`90d_std_mean_vapor_pressure_deficit_kPa`', 'std.`180d_std_precipitation_amount_mm`', 'std.`180d_std_relative_humidity_%`', 'std.`180d_std_specific_humidity_kg/kg`', 'std.`180d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`180d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`180d_std_wind_speed_m/s`', 'std.`180d_std_max_air_temperature_K`', 'std.`180d_std_min_air_temperature_K`', 'std.`180d_std_burning_index_g_Unitless`', 'std.`180d_std_dead_fuel_moisture_100hr_Percent`', 'std.`180d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`180d_std_energy_release_component-g_Unitless`', 'std.`180d_std_potential_evapotranspiration_mm`', 'std.`180d_std_mean_vapor_pressure_deficit_kPa`', 'std.`365d_std_precipitation_amount_mm`', 'std.`365d_std_relative_humidity_%`', 'std.`365d_std_specific_humidity_kg/kg`', 'std.`365d_std_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'std.`365d_std_wind_from_direction_Degrees_Clockwise_from_north`', 'std.`365d_std_wind_speed_m/s`', 'std.`365d_std_max_air_temperature_K`', 'std.`365d_std_min_air_temperature_K`', 'std.`365d_std_burning_index_g_Unitless`', 'std.`365d_std_dead_fuel_moisture_100hr_Percent`', 'std.`365d_std_dead_fuel_moisture_1000hr_Percent`', 'std.`365d_std_energy_release_component-g_Unitless`', 'std.`365d_std_potential_evapotranspiration_mm`', 'std.`365d_std_mean_vapor_pressure_deficit_kPa`']

In [31]:
import numpy as np
for pca_idx, use_cols in top5_correlates.items():
    masked_corr = corr[np.ix_(use_cols, use_cols)]
    print(masked_corr)
    print(masked_corr.tolist())
    labels = [col_names[i].split('.')[1][1:-1] for i in use_cols[:-1]] + ["PC {}".format(pca_idx - low + 1)]
    print(labels)

[[ 1.          0.99642358  0.98597737  0.9829597  -0.96460119]
 [ 0.99642358  1.          0.98447849  0.98649096 -0.96748728]
 [ 0.98597737  0.98447849  1.          0.99800091 -0.97332931]
 [ 0.9829597   0.98649096  0.99800091  1.         -0.97486828]
 [-0.96460119 -0.96748728 -0.97332931 -0.97486828  1.        ]]
[[1.0, 0.9964235843141751, 0.9859773726106559, 0.9829596966183628, -0.9646011917383406], [0.9964235843141751, 1.0, 0.9844784898650342, 0.9864909550416568, -0.9674872776125595], [0.9859773726106559, 0.9844784898650342, 1.0, 0.9980009142856752, -0.9733293052873201], [0.9829596966183628, 0.9864909550416568, 0.9980009142856752, 1.0, -0.9748682833210831], [-0.9646011917383406, -0.9674872776125595, -0.9733293052873201, -0.9748682833210831, 1.0]]
['21d_cumulative_energy_release_component-g_Unitless', '21d_mean_energy_release_component-g_Unitless', '14d_cumulative_energy_release_component-g_Unitless', '14d_mean_energy_release_component-g_Unitless', 'PC 1']
[[1.         0.99884467 0.9